<a href="https://colab.research.google.com/github/tongnet/fin7047_2026spr/blob/main/project_sample_codes" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# buy and hold strategy;
# assume picking BTC, ETH and XRP
# weights in BTC, ETH and XRP: 0.5, 0.3, 0.2
# I am using these cryptos as an illustration; your group decide what to investment and how much
# assume initial investment is $2 million

import pandas as pd
import numpy as np
import yfinance as yf

# Define the portfolio assets and initial investment
tickers = ['BTC-USD', 'ETH-USD', 'XRP-USD']
initial_investment = 2000000

# Define initial portfolio weights: 50% BTC, 30% ETH, 20% XRP
allocations = np.array([0.5, 0.3, 0.2]) * initial_investment

# Define the time period
start_date = '2020-01-01'
end_date = '2025-12-31'

# Fetch historical data
data = yf.download(tickers, start=start_date, end=end_date)['Close']

# Calculate daily returns
daily_returns = data.pct_change().dropna()

# Compute portfolio daily returns (flexible allocation)
weights = allocations / allocations.sum()
portfolio_daily_returns = daily_returns @ weights  #The @ operator in Python is the matrix multiplication (dot product) operator

# Compute total return
initial_prices = data.iloc[0]
final_prices = data.iloc[-1]

# Compute final portfolio value
final_value = (final_prices / initial_prices) * allocations
ending_balance = final_value.sum()

# Compute average daily return
avg_daily_return = portfolio_daily_returns.mean()

# Compute annualized standard deviation
annual_std_dev = portfolio_daily_returns.std() * np.sqrt(365)

# Compute Sharpe Ratio (assuming risk-free rate = 0)
sharpe_ratio = avg_daily_return * 365 / annual_std_dev

# Print results
print(f"Ending Balance: ${ending_balance:,.2f}")
print(f"Average Daily Return: {avg_daily_return:.6f}")
print(f"Annual Standard Deviation: {annual_std_dev:.6f}")
print(f"Annual Sharpe Ratio: {sharpe_ratio:.2f}")


/tmp/ipython-input-4024325730.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)['Close']
[*********************100%***********************]  3 of 3 completed

Ending Balance: $29,805,561.27
Average Daily Return: 0.002036
Annual Standard Deviation: 0.672651
Annual Sharpe Ratio: 1.10


In [5]:
# start with 100% in cash 50%/50% in bitcoin/cash when RSi > 70 and 100% in bitcoin when rsi <30
# RSI window: 14 days
# $2 million initial investments
# rebalance daily

import pandas as pd
import numpy as np
import yfinance as yf

# Get Bitcoin historical data
btc_data = yf.download("BTC-USD", start="2020-01-01", end="2025-12-31")

# Calculate daily returns
btc_data['Return'] = btc_data['Close'].pct_change()

# Define RSI
def calculate_rsi(data, window=14):    # 14 day window
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

btc_data['RSI'] = calculate_rsi(btc_data)

# Initialize portfolio
initial_investment = 2_000_000
cash = initial_investment
bitcoin = 0
portfolio_value = []

# Portfolio simulation
for i in range(1, len(btc_data)):
    rsi = btc_data['RSI'].iloc[i]
    btc_price = btc_data['Close'].iloc[i]

    # Update value of previous holdings
    cash *= 1 + (0.02 / 365)  # Daily interest on cash
    bitcoin_value = bitcoin * btc_price
    total_value = cash + bitcoin_value
    portfolio_value.append(total_value)

    # Adjust allocation
    if rsi > 70:
        target_bitcoin_value = total_value * 0.5
        target_cash_value = total_value * 0.5
    elif rsi < 30:
        target_bitcoin_value = total_value
        target_cash_value = 0
    else:
        target_bitcoin_value = 0
        target_cash_value = total_value

    # Rebalance
    bitcoin = target_bitcoin_value / btc_price
    cash = target_cash_value

# Final portfolio value
final_value = float(portfolio_value[-1])
last_date = pd.to_datetime(btc_data.index[-1]).date()
print(f"Final portfolio value on {last_date}: ${final_value:,.2f}")


/tmp/ipython-input-323629584.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc_data = yf.download("BTC-USD", start="2020-01-01", end="2025-12-31")
[*********************100%***********************]  1 of 1 completed


Final portfolio value on 2025-12-30: $15,241,843.83


/tmp/ipython-input-323629584.py:62: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  final_value = float(portfolio_value[-1])


In [ ]:
# sentiment-based investment

import pandas as pd
import yfinance as yf
import numpy as np

# Define the initial state
assets = ['BTC-USD']  # Only Bitcoin is used for this strategy
investment_amount = 2000000  # Total investment
cash_rate = 0.02  # 2% annual cash return

# Define the time period
start_date = '2020-01-01'
end_date = '2025-09-15'

# Download historical BTC data (only keep adjusted close price)
data = yf.download(assets, start=start_date, end=end_date)[['Close']]

# Load monthly sentiment index
file_path = "fear_greed_index.csv"
df = pd.read_csv(file_path)
df['date'] = pd.to_datetime(df['date'])

# Extract year and month
df['month'] = df['date'].dt.to_period('M')

# Calculate the monthly average sentiment value
monthly_sentiment = df.groupby('month')['value'].mean().reset_index()
monthly_sentiment.rename(columns={'value': 'monthly_sentiment'}, inplace=True)

# Shift the sentiment index to use prior month's value
monthly_sentiment['prior_month'] = monthly_sentiment['month'].shift(-1)
monthly_sentiment_dict = monthly_sentiment.set_index('prior_month')['monthly_sentiment'].to_dict()

# Initialize portfolio
portfolio_balance = investment_amount
btc_units = 0
cash_balance = investment_amount
position = None  # None = cash, '50% in', '100% in'
portfolio_values = []

# Iterate through BTC price data
for date, row in data.iterrows():
    month = date.to_period('M')
    avg_sentiment = monthly_sentiment_dict.get(month, np.nan)  # Retrieve sentiment or NaN if missing
    price = row['Close']

    if np.isnan(avg_sentiment):  # Skip if no sentiment data for the month
        continue

    if position is None:
        if avg_sentiment < 50:
            btc_units = portfolio_balance / price
            cash_balance = 0
            position = '100% in'
        elif avg_sentiment > 50:
            btc_units = (portfolio_balance / 2) / price
            cash_balance = portfolio_balance / 2
            position = '50% in'
    else:
        if position == '50% in' and avg_sentiment > 80:
            position = None  # Move to cash
            btc_units = 0
            cash_balance = portfolio_balance
        elif position == '100% in' and avg_sentiment < 20:
            position = None  # Move to cash
            btc_units = 0
            cash_balance = portfolio_balance
        else:
            cash_balance *= (1 + cash_rate / 365)  # Daily cash return

    # Compute new portfolio balance
    portfolio_balance = btc_units * price + cash_balance
    portfolio_values.append({'Date': date, 'Portfolio Value': portfolio_balance, 'BTC Units': btc_units, 'Cash Balance': cash_balance})

# Convert to DataFrame
portfolio_df = pd.DataFrame(portfolio_values)
portfolio_df.set_index('Date', inplace=True)

# Calculate daily returns
portfolio_df['Daily Return'] = portfolio_df['Portfolio Value'].pct_change()

# Final portfolio value
final_value = float(portfolio_value[-1])
last_date = pd.to_datetime(btc_data.index[-1]).date()
print(f"Final portfolio value on {last_date}: ${final_value:,.2f}")




C:\Users\yut3\AppData\Local\Temp\ipykernel_34760\353012296.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(assets, start=start_date, end=end_date)[['Close']]
[*********************100%***********************]  1 of 1 completed


Final portfolio value on 2025-09-14: $15,397,552.77


C:\Users\yut3\AppData\Local\Temp\ipykernel_34760\353012296.py:84: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  final_value = float(portfolio_value[-1])
